In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from glob import glob
from utils import iter_df_read

BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

In [2]:
df = pd.read_csv(f"{BASE}work_data/matriz_desercion_5_dummy_con_id.csv.gz")
df.head()

,mrun,codigo_unico,dur_estudio_carr,dur_proceso_tit,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,...,decil_50,decil_60,decil_70,decil_80,decil_90,decil_100,sub_tipo_inst_cft_estatal,sub_tipo_inst_cft_privado,sub_tipo_inst_ip_privado,sub_tipo_inst_ip_privado_precencia_nacional
0,25768831.0,I143S26C376J2V1,5,1,1,5.5,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,25768815.0,I273S1C5J2V1,6,1,0,4.7,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,25768671.0,I143S15C30J1V1,5,1,0,6.4,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,25768663.0,I116S31C51J2V1,8,0,1,5.9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,25768514.0,I143S29C17J1V1,7,1,0,5.7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
shap_best = [
    "dif_ing_ult_mat", "valor_arancel", "beca", "prom_notas_media", "gen_alu_hombre",
    "dur_proceso_tit", "edad",  "jornada_vespertino",
    "acreditada_carr_acreditada", "dur_estudio_carr",
    "dependencia_colegio_municipal", "gratuidad", "region_sede_metropolitana", 
    "decil_40", "sub_tipo_inst_ip_privado_precencia_nacional", "decil_90",
    "area_conocimiento_arte_y_arquitectura", "area_conocimiento_tecnologia",
    "jornada_diurno", 'acreditada_inst_acreditada'
]

tipo_colegio = [
    'tipo_ensenanza_colegio_hc', 'tipo_ensenanza_colegio_tp_comercial',
    'tipo_ensenanza_colegio_tp_industrial', 'tipo_ensenanza_colegio_tp_tecnica'  
]

# Matriculas 2015

In [4]:
cols_mat = [
    'cat_periodo', 'mrun', "codigo_unico", "anio_ing_carr_ori", "tipo_inst_1"
]

instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_mat)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun").drop_duplicates().reset_index(drop=True)

matriculados = matriculados[
    (matriculados["anio_ing_carr_ori"] == 2015)
]

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 491221

Cantidad de nulos: 
cat_periodo          0
codigo_unico         0
mrun                 0
anio_ing_carr_ori    0
tipo_inst_1          0
dtype: int64


,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,tipo_inst_1
0,2015,I374S1C9J1V1,5.0,2015,Centros de Formación Técnica
1,2015,I498S6C132J2V1,37.0,2015,Centros de Formación Técnica
3,2015,I111S14C309J2V1,118.0,2015,Institutos Profesionales
6,2015,I106S3C64J4V1,253.0,2015,Institutos Profesionales
8,2015,I221S1C57J1V1,370.0,2015,Centros de Formación Técnica


In [5]:
df_chico = df[shap_best + tipo_colegio + ["desertor_1", "mrun", "codigo_unico"]]

In [6]:
df_left = pd.merge(
    df_chico,
    matriculados[["codigo_unico", "mrun"]].drop_duplicates(),
    how="left"
)

df_chico.shape, matriculados.shape, df_left.shape

((731675, 27), (491221, 5), (731675, 27))

In [7]:
df_left["desertor_1"].value_counts()

0    575581
1    156094
Name: desertor_1, dtype: int64

In [8]:
df_left.drop(columns=["mrun", "codigo_unico"]).to_csv(
    f"{BASE}work_data/matriz_desercion_5_mats_2015_dep.csv.gz",
    index=False,
    compression="gzip"
)